# 扩散概率模型
## 1 基础知识
### 1.1 满足马尔可夫链的贝叶斯公式
若$A \to B \to C$，则有$P(A,B,C) = P(C|B,A)P(B,A) = P(C|B)P(B|A)P(A)$，即$P(C|B,A) = P(C|B)$。
### 1.2 高斯密度函数与叠加公式
高斯密度函数：$q(x) = \frac{1}{\sqrt{2\pi }\sigma }\exp \left ( -\frac{1}{2}\left ( \frac{x-\mu }{\sigma } \right )^2 \right )$。
\
对于两个高斯分布，
$X\sim \mathcal{N}(\mu_{1} , \sigma_{1} ^{2})$，$Y\sim \mathcal{N}(\mu_{2} , \sigma_{2} ^{2})$，叠加$aX+bY$后也为高斯分布，满足$aX+bY\sim \mathcal{N}(a\times \mu _{1} + b \times \mu_{2},a^{2} \times \sigma _{1}^{2} + b^{2} \times \sigma _{2}^{2})$。
### 1.3 KL散度
KL散度可以用来衡量两个分布的差异，假设随机变量的真实概率分布为$P$，而我们通过建模得到的一个近似分布为$Q$，则$P$与$Q$的KL散度满足下式：
<center>$D_{KL}(P,Q) = -\sum P\log Q - (-\sum P\log P) = \sum P \log \frac{P}{Q}$</center>

对于高斯分布的KL散度：$D_{KL}(p,q) = \log \frac{\sigma _{2}}{\sigma _{1}} + \frac{\sigma _{1}^{2} + (\mu _{1} - \mu _{2})^{2}}{2 \sigma _{2}^{2}} - \frac{1}{2}$。

### 1.4 参数重整化
从高斯分布$\mathcal{N}(\mu ,\sigma^{2} )$中采样可从标准分布$\mathcal{N}(0 ,1)$中采样出$z$，通过$\sigma ^{2}\ast z + \mu$实现采样过程，以实现对参数求偏导的可行性。

## 2 去噪扩散概率模型
### 2.1 Forward
扩散过程即在$t$步将原始数据逐步叠加高斯噪声，以达到破坏图像的目的。定义扩散率$\beta_{t}$，随扩散步数逐渐增大，定义扩散过程为：<center>$x_{t}=\sqrt{1-\beta _{t}}x_{t-1}+\sqrt{\beta _{t}}z _{t},\hspace{1.5em}z_{t}\sim \mathcal N(0,\boldsymbol{I})$。</center>
根据高斯过程的叠加写为概率形式：<center>$q(x_{t}\mid x_{t-1}) = \mathcal{N} (x_{t}; \sqrt{1 - \beta _{t}}x_{t-1}, \beta_{t}\boldsymbol{I})$</center>
加噪过程是马尔可夫链，写为：<center>$q(x_{1},x_{2},\cdots ,x_{T} | x_{0}) = q(x_{1} | x_{0})q(x_{2} | x_{1})\cdots q(x_{T}| x_{T-1}) = \prod_{t=1}^{T}q(x_{t}| x_{t-1})$</center>
定义$\alpha _{t} = 1 - \beta _{t}$，有：
\
$x_{t} = \sqrt{1-\beta _{t}}x_{t-1} + \sqrt{\beta _{t}}z _{t} = \sqrt{\alpha _{t}}x_{t-1}+\sqrt{\beta _{t}}z _{t}$$= \sqrt{\alpha _{t}\alpha _{t-1}\cdots \alpha _{1}}x_{0} + \sqrt{\alpha _{t}\alpha _{t-1}\cdots \alpha _{2}\beta _{1}}z _{1} + \sqrt{\alpha _{t}\alpha _{t-1}\cdots \alpha _{3}\beta _{2}}z _{2} + \cdots + \sqrt{\alpha _{t}\alpha _{t-1}\beta _{t-2}}z _{t-2} + \sqrt{\alpha _{t}\beta _{t-1}}z _{t-1} + \sqrt{\beta _{t}}z _{t}$
\
令${\bar \alpha _t} = {\alpha _t}{\alpha _{t - 1}} \cdots {\alpha _1}$，有
<center>$x_{t} = \sqrt{\bar{\alpha }_{t}}x_{0} + \sqrt{1-\bar{\alpha }_{t}}\bar{z}_{t},\hspace{1.5em}\bar{z}_{t} \sim \mathcal N(0,\boldsymbol{I})$</center>

故前向过程可以直接计算任意步的概率分布：<center>$q(x_{t}\mid x_{0}) = \mathcal{N}(x_{t}; \sqrt{\bar{\alpha }_{t}}x_{0}, (1-\bar{\alpha }_{t})\boldsymbol{I})$</center>

### 2.2 Backward
反向过程用$q(x_{t-1}\mid x_{t})$表示,从正态分布反求真实数据。若扩散率$\beta_{t}$足够小，则$q(x_{t}\mid x_{t-1})$也满足高斯分布。反向过程是通过学习一个含参$\theta$的神经网络去拟合的。
\
反向过程也是一个马尔可夫链的过程，单步有下式：
<center>$p_{\theta}(x_{t-1}\mid x_{t}) = \mathcal{N}\left ( x_{t-1}; \mu _{\theta}(x_{t}, t), \Sigma _{\theta}(x_{t}, t) \right )$</center>

有<center>$p_{\theta}(x_{0:T}) = p_{\theta}(x_{T})p_{\theta}(x_{T-1} \mid x_{T})\cdots p_{\theta}(x_{0}\mid x_{1}) = p_{\theta}(x_{T}) \prod_{t=1}^{T}p_{\theta}(x_{t-1}\mid x_{t})$</center>

将反向过程利用贝叶斯公式展开:$q(x_{t-1}\mid x_{t}) = q(x_{t}\mid x_{t-1}) \frac{q(x_{t-1})}{q(x_{t})}$,对于训练过程，已知$x_{0}$，则上式写为：<center>$q(x_{t-1}\mid x_{t}, x_{0}) = \frac{q(x_{t}\mid x_{t-1}, x_{0})\times q(x_{t-1}\mid x_{0})}{q(x_{t}\mid x_{0})} = \mathcal{N}\left ( x_{t-1}, {\tilde{\mu }(x_{t}, x_{0})}, {\tilde{\beta }_{t}}\boldsymbol{I} \right )$</center>

其中，根据高斯过程的概率密度公式可以写出：
$\tilde \mu ({x_t},{x_0}) = \frac{1}{{\sqrt {{\alpha _t}} }}\left( {{x_t} - \frac{{{\beta _t}}}{{\sqrt {1 - {{\bar \alpha }_t}} }}{{\bar z}_t}} \right)$

反向过程满足：<center>$q({x_{t - 1}}\mid {x_t},{x_0}) = {\cal N}\left( {{x_{t - 1}},\tilde \mu ({x_t},{x_0}),{{\tilde \beta }_t}{\bf{I}}} \right) = {\cal N}\left( {{x_{t - 1}},\frac{1}{{\sqrt {{\alpha _t}} }}\left( {{x_t} - \frac{{{\beta _t}}}{{\sqrt {1 - {{\bar \alpha }_t}} }}{{\bar z}_t}} \right),\frac{{1 - {{\bar \alpha }_{t - 1}}}}{{1 - {{\bar \alpha }_t}}}{\beta _t}{\bf{I}}} \right)$</center>

### 2.3 Optimize
调整参数$\theta$,使得真实数据最大化，写出极大似然函数：
$p\left ( x_{0}\mid \theta \right ) = p_{\theta }(x_{0}) = \int_{x_{1}}\int _{x_{2}}\cdots \int _{x_{T}}p_{\theta}(x_{0}, x_{1}, x_{2},\cdots ,x_{T}) d_{x_{1}}d_{x_{2}}\cdots d_{x_{T}}$$= \int_{x_{1}}\int _{x_{2}}\cdots \int _{x_{T}}{q(x_{1:T}\mid x_{0}) }\frac{p_{\theta}(x_{0}, x_{1}, x_{2},\cdots ,x_{T})}{{q(x_{1:T}\mid x_{0})}} d_{x_{1}}d_{x_{2}}\cdots d_{x_{T}}= \mathbb{E}_{ q(x_{1:T}\mid x_{0})}\left [\frac{{p_{\theta}(x_{0:T})}}{ q(x_{1:T}\mid x_{0})} \right]$

经过[推导](https://zhuanlan.zhihu.com/p/636776166?theme=dark)可得到最终优化目标：
<center>$L_{t-1}^{simple}= \mathbb{E}_{ x_{0}, \bar{z}_{t} \sim \mathcal{N}(0,\boldsymbol{I})} \left [ \left \| \bar{z}_{t} - z_{\theta}(\sqrt{\bar{\alpha }_{t}}x_{0} + \sqrt{1-\bar{\alpha }_{t}}\bar{z}_{t}, t) \right \| ^{2} \right ]$</center>

最终得到的优化目标非常简单，就是让网络预测的噪声与真实的噪声一致。

### 2.4 优化的DDPM（未阅读）
来源[Improved Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2102.09672)，利用变化的方差和非线性变化的扩散率以提高生成网络的有效性。


## 3 TimeGrad（DDPM）
Timegrad是使用DDPM进行时间序列概率预测的方法，主要思想是将传统的RNN网络和DDPM结合起来，将RNN的隐藏状态作为DDPM的输入生成$x_t^0$，从而预测时间序列。
### 3.0 参考资料
原论文[Autoregressive Denoising Diffusion Models for Multivariate Probabilistic Time Series Forecasting](https://arxiv.org/abs/2101.12072)。

笔记[Autoregressive Denoising Diffusion Model](https://datumorphism.leima.is/wiki/time-series/autoregressive-denoising-diffusion-model/)。
### 3.1 网络架构
#### 3.1.1 整体架构
![img](https://github.com/Gins-X/AI/blob/main/img/1.png?raw=true)
#### 3.1.2 噪声预测网络结构
利用8级残差网络拟合噪声。
![img2](https://github.com/Gins-X/AI/blob/main/img/2.PNG?raw=true)
### 3.2 概率预测
由于扩散模型基于标准高斯分布采样，生成的预测值也将服从一定的分布而非固定，多次生成，利用分位数去代替最终的概率分布。**若需要平滑绘制曲线，可以考虑插值再绘制，插值方法也许可以参考CSDI**。
```python
ps_data = [forecast.quantile(p / 100.0)[:,dim] for p in percentiles_sorted]
i_p50 = len(percentiles_sorted) // 2
#将中位数作为预测均值        
p50_data = ps_data[i_p50]
p50_series = pd.Series(data=p50_data, index=forecast.index)
p50_series.plot(color=color, ls="-", label=f"{label_prefix}median", ax=ax)     
#用分位数代替概率预测结果
ax.fill_between(
    forecast.index,
    ps_data[i],
    ps_data[-i - 1],
    facecolor=color,
    alpha=alpha,
    interpolate=True,
)
```
### 3.3 核心公式
所需优化的LOSS函数：<center>$\mathcal L_t = \mathbb E_{\mathbf x^0_t, \epsilon, n} \left[ \lVert \epsilon - \epsilon_\theta ( \sqrt{\bar \alpha_n} \mathbf x^0_t + \sqrt{1-\bar \alpha_n}\epsilon, \mathbf h_{t-1}, n ) \rVert^2 \right]$</center>
预测公式：
<center>$\mathbf x^{n-1}_{T+1} = \frac{1}{\alpha_n} \left( \mathbf x^n_{T+1} - \frac{\beta_n}{1 - \bar\alpha_n} \epsilon_\theta( \mathbf x^n_{T+1}, \mathbf h_{T}, n ) \right) + \sqrt{\Sigma_\theta} \mathbf z$</center>

## 4 Score-Based Diffusion Model
论文来源：[CSDI: Conditional Score-based Diffusion Models for Probabilistic Time Series Imputation](https://arxiv.org/abs/2107.03502)